In [ ]:
"""
Transcription Factor Binding Motif (TFBM) Library Generator
============================================================

JASPARデータベースとHT-SELEX論文に基づくTFBMライブラリ設計システム

【設計原則】
基準1: JASPAR - ユビキタス発現遺伝子のプロモーターから選抜
        HT-SELEX - 高発現予測モチーフを選抜 (Jolma et al. 2013)
基準2: 同じTFBMを4コピーと6コピー含むモジュール
基準3: 10bp spacer (同一側面) / 5bp spacer (反対側面)

【拡張性】
- 新規モチーフの追加が容易
- Core Promoterシステムとの統合API
- フィルタリング・選抜機能

References:
-----------
- Jolma et al. (2013) Cell 152:327-339
- Zahm et al. (2024) Nature Communications 15:10353
- JASPAR 2024 database
"""

import pandas as pd
from typing import Dict, List, Tuple, Optional, Union
from dataclasses import dataclass, field
from enum import Enum
import json
from datetime import datetime


# ============================================================================
# Enums and Data Classes
# ============================================================================

class MotifSource(Enum):
    """モチーフデータソース"""
    JASPAR = "JASPAR"
    HT_SELEX = "HT-SELEX"
    LITERATURE = "Literature"
    CUSTOM = "Custom"


class ExpressionType(Enum):
    """発現パターン"""
    UBIQUITOUS = "Ubiquitous"          # ユビキタス
    TISSUE_SPECIFIC = "Tissue-specific" # 組織特異的
    INDUCIBLE = "Inducible"             # 誘導性
    CONSTITUTIVE = "Constitutive"       # 恒常的


class HelixPosition(Enum):
    """DNA二重鎖上の位置"""
    SAME_SIDE = "same_side"         # 同一側面 (10 bp spacer)
    OPPOSITE_SIDE = "opposite_side"  # 反対側面 (5 bp spacer)


@dataclass
class TranscriptionFactorMotif:
    """
    転写因子結合モチーフの定義クラス
    
    拡張可能な設計: 新しいモチーフは辞書形式で追加可能
    """
    tf_name: str                    # 転写因子名
    motif_id: str                   # ユニークID
    consensus_sequence: str          # コンセンサス配列
    optimal_sequence: str            # 最適配列（縮重解決済み）
    source: MotifSource             # データソース
    expression_type: ExpressionType # 発現パターン
    tf_family: str                  # TFファミリー
    relative_activity: float        # 相対活性 (1.0 = baseline)
    target_genes: List[str]         # 標的遺伝子例
    notes: str                      # 備考
    reference: str                  # 参考文献
    jaspar_matrix_id: Optional[str] = None  # JASPAR Matrix ID
    ht_selex_enrichment: Optional[float] = None  # HT-SELEX enrichment


# ============================================================================
# JASPAR Database - Ubiquitous Expression Motifs
# ============================================================================

class JASPARMotifDatabase:
    """
    JASPAR 2024ベースのユビキタス発現関連モチーフデータベース
    
    選抜基準:
    - 広い細胞種で発現する遺伝子のプロモーターに存在
    - ハウスキーピング遺伝子の発現制御に関与
    - 高品質スコア (Quality A or B)
    """
    
    def __init__(self):
        self.motifs: Dict[str, TranscriptionFactorMotif] = {}
        self._initialize_motifs()
    
    def _initialize_motifs(self):
        """JASPARモチーフの初期化"""
        
        # ==================== SP/KLF Family (GC-box) ====================
        self.motifs['SP1'] = TranscriptionFactorMotif(
            tf_name='SP1',
            motif_id='MA0079.5',
            consensus_sequence='GGGCGGG',
            optimal_sequence='GGGCGGGCG',
            source=MotifSource.JASPAR,
            expression_type=ExpressionType.UBIQUITOUS,
            tf_family='C2H2-ZF (SP/KLF)',
            relative_activity=1.5,
            target_genes=['ACTB', 'GAPDH', 'DHFR', 'HPRT1'],
            notes='GC box, housekeeping promoters, strongest SP family',
            reference='JASPAR 2024; Suske (1999)',
            jaspar_matrix_id='MA0079.5'
        )
        
        self.motifs['KLF4'] = TranscriptionFactorMotif(
            tf_name='KLF4',
            motif_id='MA0039.4',
            consensus_sequence='CACCC',
            optimal_sequence='AGGGTGTGGC',
            source=MotifSource.JASPAR,
            expression_type=ExpressionType.UBIQUITOUS,
            tf_family='C2H2-ZF (SP/KLF)',
            relative_activity=1.3,
            target_genes=['NANOG', 'OCT4', 'MYC'],
            notes='Pluripotency factor, CACCC box',
            reference='JASPAR 2024; Takahashi & Yamanaka (2006)',
            jaspar_matrix_id='MA0039.4'
        )
        
        # ==================== NF-Y Family (CCAAT-box) ====================
        self.motifs['NFY'] = TranscriptionFactorMotif(
            tf_name='NF-Y',
            motif_id='MA0060.3',
            consensus_sequence='CCAAT',
            optimal_sequence='GATTGGCTGATTGG',
            source=MotifSource.JASPAR,
            expression_type=ExpressionType.UBIQUITOUS,
            tf_family='NF-Y/CBF',
            relative_activity=1.4,
            target_genes=['HSP70', 'MHC-II', 'Histone genes'],
            notes='CCAAT box, 70% of eukaryotic promoters',
            reference='JASPAR 2024; Mantovani (1999)',
            jaspar_matrix_id='MA0060.3'
        )
        
        # ==================== E2F Family ====================
        self.motifs['E2F1'] = TranscriptionFactorMotif(
            tf_name='E2F1',
            motif_id='MA0024.3',
            consensus_sequence='TTTCGCGC',
            optimal_sequence='TTTCGCGCGAAA',
            source=MotifSource.JASPAR,
            expression_type=ExpressionType.CONSTITUTIVE,
            tf_family='E2F',
            relative_activity=1.6,
            target_genes=['CCNE1', 'CDC6', 'MCM genes'],
            notes='Cell cycle S-phase genes',
            reference='JASPAR 2024; Dyson (1998)',
            jaspar_matrix_id='MA0024.3'
        )
        
        # ==================== ETS Family ====================
        self.motifs['ETS1'] = TranscriptionFactorMotif(
            tf_name='ETS1',
            motif_id='MA0098.3',
            consensus_sequence='GGAA',
            optimal_sequence='ACCGGAAGT',
            source=MotifSource.JASPAR,
            expression_type=ExpressionType.UBIQUITOUS,
            tf_family='ETS',
            relative_activity=1.4,
            target_genes=['MMP genes', 'VEGFR', 'Integrins'],
            notes='ETS core GGA(A/T), Class I',
            reference='JASPAR 2024; Wei et al. (2010)',
            jaspar_matrix_id='MA0098.3'
        )
        
        self.motifs['ELK1'] = TranscriptionFactorMotif(
            tf_name='ELK1',
            motif_id='MA0028.2',
            consensus_sequence='CCGGAA',
            optimal_sequence='ACCCGGAAGT',
            source=MotifSource.JASPAR,
            expression_type=ExpressionType.INDUCIBLE,
            tf_family='ETS (TCF)',
            relative_activity=1.5,
            target_genes=['FOS', 'EGR1', 'JUN'],
            notes='TCF subfamily, immediate early genes',
            reference='JASPAR 2024; Sharrocks (2001)',
            jaspar_matrix_id='MA0028.2'
        )
        
        self.motifs['GABPA'] = TranscriptionFactorMotif(
            tf_name='GABPA',
            motif_id='MA0062.3',
            consensus_sequence='CGGAAG',
            optimal_sequence='ACCGGAAGTG',
            source=MotifSource.JASPAR,
            expression_type=ExpressionType.UBIQUITOUS,
            tf_family='ETS',
            relative_activity=1.3,
            target_genes=['COX genes', 'ATP synthase'],
            notes='NRF-2α, mitochondrial biogenesis',
            reference='JASPAR 2024; Scarpulla (2008)',
            jaspar_matrix_id='MA0062.3'
        )
        
        # ==================== bHLH Family (E-box) ====================
        self.motifs['USF1'] = TranscriptionFactorMotif(
            tf_name='USF1',
            motif_id='MA0093.3',
            consensus_sequence='CACGTG',
            optimal_sequence='GTCACGTGAC',
            source=MotifSource.JASPAR,
            expression_type=ExpressionType.UBIQUITOUS,
            tf_family='bHLH',
            relative_activity=1.4,
            target_genes=['FAS', 'Lipogenic genes'],
            notes='E-box (CACGTG), metabolic genes',
            reference='JASPAR 2024; Corre & Bhattacharjee (2009)',
            jaspar_matrix_id='MA0093.3'
        )
        
        self.motifs['MYC'] = TranscriptionFactorMotif(
            tf_name='MYC',
            motif_id='MA0147.3',
            consensus_sequence='CACGTG',
            optimal_sequence='CCACGTGGT',
            source=MotifSource.JASPAR,
            expression_type=ExpressionType.CONSTITUTIVE,
            tf_family='bHLH-LZ',
            relative_activity=1.8,
            target_genes=['ODC1', 'LDHA', 'Ribosomal genes'],
            notes='E-box, very strong, proliferation',
            reference='JASPAR 2024; Dang (2012)',
            jaspar_matrix_id='MA0147.3'
        )
        
        # ==================== bZIP Family ====================
        self.motifs['CREB1'] = TranscriptionFactorMotif(
            tf_name='CREB1',
            motif_id='MA0018.4',
            consensus_sequence='TGACGTCA',
            optimal_sequence='TGACGTCA',
            source=MotifSource.JASPAR,
            expression_type=ExpressionType.INDUCIBLE,
            tf_family='bZIP (CREB/ATF)',
            relative_activity=1.5,
            target_genes=['FOS', 'BDNF', 'Somatostatin'],
            notes='CRE element, cAMP-inducible',
            reference='JASPAR 2024; Mayr & Montminy (2001)',
            jaspar_matrix_id='MA0018.4'
        )
        
        self.motifs['ATF4'] = TranscriptionFactorMotif(
            tf_name='ATF4',
            motif_id='MA0833.2',
            consensus_sequence='TGATGCAA',
            optimal_sequence='TTGCATCA',
            source=MotifSource.JASPAR,
            expression_type=ExpressionType.INDUCIBLE,
            tf_family='bZIP (CREB/ATF)',
            relative_activity=1.3,
            target_genes=['CHOP', 'ASNS', 'ATF3'],
            notes='Integrated stress response',
            reference='JASPAR 2024; Pakos-Zebrucka (2016)',
            jaspar_matrix_id='MA0833.2'
        )
        
        self.motifs['JUN'] = TranscriptionFactorMotif(
            tf_name='JUN',
            motif_id='MA0488.1',
            consensus_sequence='TGAGTCA',
            optimal_sequence='ATGAGTCAT',
            source=MotifSource.JASPAR,
            expression_type=ExpressionType.INDUCIBLE,
            tf_family='bZIP (AP-1)',
            relative_activity=1.6,
            target_genes=['MMP1', 'IL2', 'Cyclin D1'],
            notes='TRE element, AP-1 site',
            reference='JASPAR 2024; Shaulian & Karin (2002)',
            jaspar_matrix_id='MA0488.1'
        )
        
        # ==================== NRF1 ====================
        self.motifs['NRF1'] = TranscriptionFactorMotif(
            tf_name='NRF1',
            motif_id='MA0506.1',
            consensus_sequence='GCGCATGCGC',
            optimal_sequence='GCGCATGCGC',
            source=MotifSource.JASPAR,
            expression_type=ExpressionType.UBIQUITOUS,
            tf_family='NRF',
            relative_activity=1.4,
            target_genes=['TFAM', 'COX genes', 'mtDNA'],
            notes='Mitochondrial gene expression',
            reference='JASPAR 2024; Scarpulla (2011)',
            jaspar_matrix_id='MA0506.1'
        )
        
        # ==================== YY1 ====================
        self.motifs['YY1'] = TranscriptionFactorMotif(
            tf_name='YY1',
            motif_id='MA0095.2',
            consensus_sequence='CAAATGG',
            optimal_sequence='CCGCCATATT',
            source=MotifSource.JASPAR,
            expression_type=ExpressionType.UBIQUITOUS,
            tf_family='C2H2-ZF (GLI-like)',
            relative_activity=1.3,
            target_genes=['Ribosomal proteins', 'MYC'],
            notes='Bidirectional regulator',
            reference='JASPAR 2024; Gordon et al. (2006)',
            jaspar_matrix_id='MA0095.2'
        )
        
        # ==================== NFKB ====================
        self.motifs['NFKB1'] = TranscriptionFactorMotif(
            tf_name='NFKB1',
            motif_id='MA0105.4',
            consensus_sequence='GGGAATTTCC',
            optimal_sequence='GGGAATTTCC',
            source=MotifSource.JASPAR,
            expression_type=ExpressionType.INDUCIBLE,
            tf_family='RHD',
            relative_activity=1.7,
            target_genes=['IL6', 'TNF', 'Immune genes'],
            notes='NFκB p50 subunit',
            reference='JASPAR 2024',
            jaspar_matrix_id='MA0105.4'
        )
    
    def get_motif(self, name: str) -> Optional[TranscriptionFactorMotif]:
        return self.motifs.get(name)
    
    def get_all_motifs(self) -> List[TranscriptionFactorMotif]:
        return list(self.motifs.values())
    
    def get_motifs_by_family(self, family: str) -> List[TranscriptionFactorMotif]:
        return [m for m in self.motifs.values() if family.lower() in m.tf_family.lower()]
    
    def get_ubiquitous_motifs(self) -> List[TranscriptionFactorMotif]:
        return [m for m in self.motifs.values() 
                if m.expression_type == ExpressionType.UBIQUITOUS]
    
    def add_motif(self, name: str, motif: TranscriptionFactorMotif):
        """新規モチーフ追加（拡張用API）"""
        self.motifs[name] = motif
    
    def list_available_motifs(self) -> List[str]:
        """利用可能なモチーフ名一覧"""
        return list(self.motifs.keys())


# ============================================================================
# HT-SELEX Database - High Expression Motifs
# ============================================================================

class HTSELEXMotifDatabase:
    """
    HT-SELEX (Jolma et al. 2013)ベースの高発現モチーフデータベース
    
    選抜基準:
    - 高いenrichment score
    - 明確なコンセンサス配列
    - Zahm et al. (2024)でMPRA検証済みを優先
    """
    
    def __init__(self):
        self.motifs: Dict[str, TranscriptionFactorMotif] = {}
        self._initialize_motifs()
    
    def _initialize_motifs(self):
        """HT-SELEXモチーフの初期化"""
        
        # ==================== ETS Class I (Highest enrichment) ====================
        self.motifs['ETS_ClassI'] = TranscriptionFactorMotif(
            tf_name='ETS Class I',
            motif_id='SELEX_ETS1',
            consensus_sequence='ACCGGAAGT',
            optimal_sequence='ACCGGAAGT',
            source=MotifSource.HT_SELEX,
            expression_type=ExpressionType.UBIQUITOUS,
            tf_family='ETS',
            relative_activity=1.8,
            target_genes=['ERG', 'FLI1', 'ETS1'],
            notes='Strongest ETS enrichment',
            reference='Jolma et al. (2013) Cell',
            ht_selex_enrichment=2.0
        )
        
        # ERG dimeric site
        self.motifs['ERG_dimer'] = TranscriptionFactorMotif(
            tf_name='ERG dimer',
            motif_id='SELEX_ERG_dim',
            consensus_sequence='ACCGGAAGTGGCGGAA',
            optimal_sequence='ACCGGAAGTGGCGGAA',
            source=MotifSource.HT_SELEX,
            expression_type=ExpressionType.UBIQUITOUS,
            tf_family='ETS',
            relative_activity=2.0,
            target_genes=['Prostate genes', 'Endothelial genes'],
            notes='Dimeric ETS, head-to-head',
            reference='Jolma et al. (2013)',
            ht_selex_enrichment=2.2
        )
        
        # ==================== bHLH E-box variants ====================
        self.motifs['EBOX_canonical'] = TranscriptionFactorMotif(
            tf_name='E-box Canonical',
            motif_id='SELEX_EBOX1',
            consensus_sequence='CACGTG',
            optimal_sequence='GCCACGTGGC',
            source=MotifSource.HT_SELEX,
            expression_type=ExpressionType.UBIQUITOUS,
            tf_family='bHLH',
            relative_activity=1.7,
            target_genes=['MYC targets', 'USF targets'],
            notes='Canonical E-box, highest bHLH enrichment',
            reference='Jolma et al. (2013)',
            ht_selex_enrichment=1.8
        )
        
        self.motifs['EBOX_CAG'] = TranscriptionFactorMotif(
            tf_name='E-box CAG',
            motif_id='SELEX_EBOX_CAG',
            consensus_sequence='CAGCTG',
            optimal_sequence='GCAGCTGCC',
            source=MotifSource.HT_SELEX,
            expression_type=ExpressionType.UBIQUITOUS,
            tf_family='bHLH',
            relative_activity=1.5,
            target_genes=['MYOD targets', 'Muscle genes'],
            notes='E-box variant',
            reference='Jolma et al. (2013)',
            ht_selex_enrichment=1.6
        )
        
        # ==================== bZIP CRE/TRE ====================
        self.motifs['CRE_optimal'] = TranscriptionFactorMotif(
            tf_name='CRE Optimal',
            motif_id='SELEX_CRE',
            consensus_sequence='TGACGTCA',
            optimal_sequence='TGACGTCA',
            source=MotifSource.HT_SELEX,
            expression_type=ExpressionType.INDUCIBLE,
            tf_family='bZIP (CREB/ATF)',
            relative_activity=1.8,
            target_genes=['FOS', 'VIP', 'Somatostatin'],
            notes='Perfect CRE palindrome',
            reference='Jolma et al. (2013); Zahm et al. (2024)',
            ht_selex_enrichment=1.9
        )
        
        self.motifs['TRE_optimal'] = TranscriptionFactorMotif(
            tf_name='TRE Optimal',
            motif_id='SELEX_TRE',
            consensus_sequence='TGAGTCA',
            optimal_sequence='ATGAGTCAT',
            source=MotifSource.HT_SELEX,
            expression_type=ExpressionType.INDUCIBLE,
            tf_family='bZIP (AP-1)',
            relative_activity=1.7,
            target_genes=['MMP1', 'IL2'],
            notes='AP-1/TRE site',
            reference='Jolma et al. (2013)',
            ht_selex_enrichment=1.8
        )
        
        # MAFB (Zahm 2024: forskolin responsive)
        self.motifs['MAFB'] = TranscriptionFactorMotif(
            tf_name='MAFB',
            motif_id='SELEX_MAFB',
            consensus_sequence='TGCTGACGTAAGCA',
            optimal_sequence='TGCTGACGTAAGCA',
            source=MotifSource.HT_SELEX,
            expression_type=ExpressionType.INDUCIBLE,
            tf_family='bZIP (MAF)',
            relative_activity=1.9,
            target_genes=['Insulin', 'Glucagon'],
            notes='CRE-like core, forskolin-responsive (Zahm 2024)',
            reference='Jolma et al. (2013); Zahm et al. (2024)',
            ht_selex_enrichment=2.0
        )
        
        # ==================== SRF (Serum Response) ====================
        self.motifs['SRF'] = TranscriptionFactorMotif(
            tf_name='SRF',
            motif_id='SELEX_SRF',
            consensus_sequence='CCATATTAGG',
            optimal_sequence='CCATATTAGG',
            source=MotifSource.HT_SELEX,
            expression_type=ExpressionType.INDUCIBLE,
            tf_family='MADS-Box',
            relative_activity=1.8,
            target_genes=['FOS', 'ACTA2', 'Immediate early genes'],
            notes='CArG box, serum response element',
            reference='Jolma et al. (2013); Zahm et al. (2024)',
            ht_selex_enrichment=1.9
        )
        
        # ==================== THRB-1 (Zahm 2024: serum/Gαq response) ====================
        self.motifs['THRB1'] = TranscriptionFactorMotif(
            tf_name='THRB-1',
            motif_id='SELEX_THRB1',
            consensus_sequence='TGACCTTATAAGGTCA',
            optimal_sequence='TGACCTTATAAGGTCA',
            source=MotifSource.HT_SELEX,
            expression_type=ExpressionType.INDUCIBLE,
            tf_family='Nuclear Receptor',
            relative_activity=1.6,
            target_genes=['Thyroid hormone responsive genes'],
            notes='Palindromic DR4, serum/SRF responsive (Zahm 2024)',
            reference='Jolma et al. (2013); Zahm et al. (2024)',
            ht_selex_enrichment=1.7
        )
        
        # ==================== p53 Family ====================
        self.motifs['TP53'] = TranscriptionFactorMotif(
            tf_name='p53',
            motif_id='SELEX_TP53',
            consensus_sequence='GGGCATGCCC',
            optimal_sequence='GGGCATGCCC',
            source=MotifSource.HT_SELEX,
            expression_type=ExpressionType.INDUCIBLE,
            tf_family='p53',
            relative_activity=1.5,
            target_genes=['CDKN1A', 'BAX', 'MDM2'],
            notes='p53 half-site, stress responsive',
            reference='Jolma et al. (2013)',
            ht_selex_enrichment=1.6
        )
        
        # ==================== NFAT Family ====================
        self.motifs['NFAT'] = TranscriptionFactorMotif(
            tf_name='NFAT',
            motif_id='SELEX_NFAT',
            consensus_sequence='TGGAAAAT',
            optimal_sequence='TGGAAAAT',
            source=MotifSource.HT_SELEX,
            expression_type=ExpressionType.INDUCIBLE,
            tf_family='RHD',
            relative_activity=1.4,
            target_genes=['IL2', 'IL4', 'TNF'],
            notes='NFAT5 activated by LiCl (Zahm 2024)',
            reference='Jolma et al. (2013); Zahm et al. (2024)',
            ht_selex_enrichment=1.5
        )
        
        # ==================== Metal Response ====================
        self.motifs['MRE'] = TranscriptionFactorMotif(
            tf_name='MTF1 (MRE)',
            motif_id='SELEX_MRE',
            consensus_sequence='TGCACAC',
            optimal_sequence='TGCACAC',
            source=MotifSource.HT_SELEX,
            expression_type=ExpressionType.INDUCIBLE,
            tf_family='C2H2-ZF',
            relative_activity=1.5,
            target_genes=['MT1A', 'MT2A', 'Metallothioneins'],
            notes='Metal response element, Zn/Cd responsive',
            reference='Jolma et al. (2013); Zahm et al. (2024)',
            ht_selex_enrichment=1.6
        )
        
        # ==================== HSF (Heat Shock) ====================
        self.motifs['HSF1'] = TranscriptionFactorMotif(
            tf_name='HSF1',
            motif_id='SELEX_HSF1',
            consensus_sequence='GAAGCTTC',
            optimal_sequence='GAAGCTTCGAAG',
            source=MotifSource.HT_SELEX,
            expression_type=ExpressionType.INDUCIBLE,
            tf_family='HSF',
            relative_activity=1.6,
            target_genes=['HSP70', 'HSP90', 'Chaperones'],
            notes='Heat shock element, stress responsive',
            reference='Jolma et al. (2013)',
            ht_selex_enrichment=1.7
        )
        
        # ==================== Homeodomain ====================
        self.motifs['HOX_TAAT'] = TranscriptionFactorMotif(
            tf_name='Homeodomain TAAT',
            motif_id='SELEX_HOX_TAAT',
            consensus_sequence='TAATTA',
            optimal_sequence='CTAATTAG',
            source=MotifSource.HT_SELEX,
            expression_type=ExpressionType.UBIQUITOUS,
            tf_family='Homeodomain',
            relative_activity=1.4,
            target_genes=['Development genes'],
            notes='Core homeodomain site',
            reference='Jolma et al. (2013)',
            ht_selex_enrichment=1.5
        )
        
        # ==================== NFKB ====================
        self.motifs['NFKB'] = TranscriptionFactorMotif(
            tf_name='NF-κB',
            motif_id='SELEX_NFKB',
            consensus_sequence='GGGAATTTCC',
            optimal_sequence='GGGAATTTCC',
            source=MotifSource.HT_SELEX,
            expression_type=ExpressionType.INDUCIBLE,
            tf_family='RHD',
            relative_activity=1.7,
            target_genes=['IL6', 'TNF', 'Immune genes'],
            notes='NF-κB consensus',
            reference='Jolma et al. (2013)',
            ht_selex_enrichment=1.8
        )
    
    def get_motif(self, name: str) -> Optional[TranscriptionFactorMotif]:
        return self.motifs.get(name)
    
    def get_all_motifs(self) -> List[TranscriptionFactorMotif]:
        return list(self.motifs.values())
    
    def get_high_enrichment_motifs(self, threshold: float = 1.5) -> List[TranscriptionFactorMotif]:
        return [m for m in self.motifs.values() 
                if m.ht_selex_enrichment and m.ht_selex_enrichment >= threshold]
    
    def add_motif(self, name: str, motif: TranscriptionFactorMotif):
        """新規モチーフ追加（拡張用API）"""
        self.motifs[name] = motif
    
    def list_available_motifs(self) -> List[str]:
        """利用可能なモチーフ名一覧"""
        return list(self.motifs.keys())


# ============================================================================
# TFBM Module Generator
# ============================================================================

class TFBMModuleGenerator:
    """
    TFBMモジュール生成クラス
    
    基準2: 4コピー / 6コピー
    基準3: 10 bp spacer (同一側面) / 5 bp spacer (反対側面)
    """
    
    # DNA二重鎖の1回転 ≈ 10.5 bp
    DNA_HELIX_TURN = 10.5
    
    def __init__(self):
        self.jaspar_db = JASPARMotifDatabase()
        self.htselex_db = HTSELEXMotifDatabase()
        
        # スペーサー設定
        self.SAME_SIDE_SPACER = 10      # DNA二重鎖の同じ側面
        self.OPPOSITE_SIDE_SPACER = 5    # DNA二重鎖の反対側面
        
        # コピー数設定
        self.COPY_NUMBERS = [4, 6]
        
        # 最適化スペーサー配列（GC含量~45%、3連続回避）
        self.spacer_sequences = {
            5: 'ATCGA',         # GC=40%
            10: 'CTAGCTAGCT',   # GC=40%
        }
    
    def _get_spacer_sequence(self, length: int) -> str:
        """スペーサー配列を取得"""
        if length in self.spacer_sequences:
            return self.spacer_sequences[length]
        
        # デフォルトパターン
        pattern = 'CTAGCTAGCT'
        repeats = (length // len(pattern)) + 1
        return (pattern * repeats)[:length]
    
    def _generate_module(self, 
                         motif: TranscriptionFactorMotif,
                         copy_number: int,
                         spacer_length: int) -> Dict:
        """
        単一TFBMモジュールの生成
        
        Parameters:
        -----------
        motif : TranscriptionFactorMotif
        copy_number : int (4 or 6)
        spacer_length : int (5 or 10 bp)
        
        Returns:
        --------
        Dict : モジュール情報
        """
        spacer = self._get_spacer_sequence(spacer_length)
        
        # モチーフの繰り返し配列を構築
        parts = []
        for i in range(copy_number):
            parts.append(motif.optimal_sequence)
            if i < copy_number - 1:
                parts.append(spacer)
        
        full_sequence = ''.join(parts)
        
        # Configuration名
        side_config = "same_side" if spacer_length == 10 else "opposite_side"
        
        # GC含量計算
        gc_count = full_sequence.count('G') + full_sequence.count('C')
        gc_content = round(gc_count / len(full_sequence) * 100, 1)
        
        return {
            'module_id': f"TFBM_{motif.motif_id}_{copy_number}x_{spacer_length}bp",
            'config_name': f"{motif.tf_name}_{copy_number}x_{side_config}",
            'tf_name': motif.tf_name,
            'tf_family': motif.tf_family,
            'motif_source': motif.source.value,
            'consensus_sequence': motif.consensus_sequence,
            'optimal_sequence': motif.optimal_sequence,
            'motif_length': len(motif.optimal_sequence),
            'copy_number': copy_number,
            'spacer_length': spacer_length,
            'spacer_sequence': spacer,
            'helix_position': side_config,
            'full_sequence': full_sequence,
            'total_length': len(full_sequence),
            'gc_content': gc_content,
            'expression_type': motif.expression_type.value,
            'relative_activity': motif.relative_activity,
            'reference': motif.reference,
            'notes': motif.notes,
        }
    
    def generate_library(self,
                         include_jaspar: bool = True,
                         include_htselex: bool = True,
                         jaspar_filter: Optional[str] = None,
                         htselex_min_enrichment: float = 1.0) -> pd.DataFrame:
        """
        TFBMライブラリの生成
        
        Parameters:
        -----------
        include_jaspar : bool
            JASPARモチーフを含める
        include_htselex : bool
            HT-SELEXモチーフを含める
        jaspar_filter : str, optional
            JASPARファミリーフィルタ
        htselex_min_enrichment : float
            HT-SELEX最小enrichmentスコア
        
        Returns:
        --------
        pd.DataFrame : TFBMライブラリ
        """
        all_modules = []
        
        # JASPARモチーフ処理
        if include_jaspar:
            jaspar_motifs = self.jaspar_db.get_all_motifs()
            if jaspar_filter:
                jaspar_motifs = self.jaspar_db.get_motifs_by_family(jaspar_filter)
            
            for motif in jaspar_motifs:
                for copy_num in self.COPY_NUMBERS:
                    for spacer_len in [self.SAME_SIDE_SPACER, self.OPPOSITE_SIDE_SPACER]:
                        module = self._generate_module(motif, copy_num, spacer_len)
                        all_modules.append(module)
        
        # HT-SELEXモチーフ処理
        if include_htselex:
            htselex_motifs = self.htselex_db.get_all_motifs()
            if htselex_min_enrichment > 1.0:
                htselex_motifs = self.htselex_db.get_high_enrichment_motifs(htselex_min_enrichment)
            
            for motif in htselex_motifs:
                for copy_num in self.COPY_NUMBERS:
                    for spacer_len in [self.SAME_SIDE_SPACER, self.OPPOSITE_SIDE_SPACER]:
                        module = self._generate_module(motif, copy_num, spacer_len)
                        all_modules.append(module)
        
        return pd.DataFrame(all_modules)
    
    def get_summary(self, df: pd.DataFrame) -> Dict:
        """ライブラリサマリー生成"""
        return {
            'total_modules': len(df),
            'unique_tfs': df['tf_name'].nunique(),
            'by_source': df['motif_source'].value_counts().to_dict(),
            'by_copy_number': df['copy_number'].value_counts().to_dict(),
            'by_spacer': df['spacer_length'].value_counts().to_dict(),
            'by_family': df['tf_family'].value_counts().to_dict(),
            'by_expression_type': df['expression_type'].value_counts().to_dict(),
            'length_stats': {
                'min': df['total_length'].min(),
                'max': df['total_length'].max(),
                'mean': round(df['total_length'].mean(), 1),
            },
            'gc_content_stats': {
                'min': df['gc_content'].min(),
                'max': df['gc_content'].max(),
                'mean': round(df['gc_content'].mean(), 1),
            }
        }


# ============================================================================
# Integration with Core Promoter System
# ============================================================================

class IntegratedPromoterLibrary:
    """
    Core PromoterとTFBMの統合ライブラリ生成
    
    完全長プロモーター構造:
    [TFBM Module] -- [Connector] -- [Core Promoter]
    """
    
    def __init__(self, core_promoter_df: Optional[pd.DataFrame] = None):
        self.tfbm_generator = TFBMModuleGenerator()
        self.core_promoter_df = core_promoter_df
        
        # 接続用スペーサー
        self.CONNECTOR_SPACER = 'CTAGCTAG'  # 8 bp
    
    def load_core_promoters(self, filepath: str):
        """Core PromoterをCSVから読み込み"""
        self.core_promoter_df = pd.read_csv(filepath)
        print(f"Loaded {len(self.core_promoter_df)} core promoters from {filepath}")
    
    def generate_tfbm_library(self, **kwargs) -> pd.DataFrame:
        """TFBMライブラリ生成"""
        return self.tfbm_generator.generate_library(**kwargs)
    
    def combine_with_core_promoter(self,
                                   tfbm_df: pd.DataFrame,
                                   core_promoter_df: Optional[pd.DataFrame] = None,
                                   selected_core_ids: Optional[List[str]] = None
                                  ) -> pd.DataFrame:
        """
        TFBMとCore Promoterを統合
        
        Parameters:
        -----------
        tfbm_df : pd.DataFrame
            TFBMライブラリ
        core_promoter_df : pd.DataFrame, optional
            Core Promoterライブラリ
        selected_core_ids : List[str], optional
            使用するCore Promoter IDリスト
        
        Returns:
        --------
        pd.DataFrame : 統合プロモーターライブラリ
        """
        if core_promoter_df is None:
            core_promoter_df = self.core_promoter_df
        
        if core_promoter_df is None:
            print("Error: No core promoter data available")
            return tfbm_df
        
        if selected_core_ids:
            core_df = core_promoter_df[core_promoter_df['ID'].isin(selected_core_ids)]
        else:
            core_df = core_promoter_df
        
        combined = []
        
        for _, tfbm in tfbm_df.iterrows():
            for _, core in core_df.iterrows():
                full_seq = tfbm['full_sequence'] + self.CONNECTOR_SPACER + core['Sequence']
                
                combined.append({
                    'promoter_id': f"{tfbm['module_id']}_{core['ID']}",
                    'tfbm_id': tfbm['module_id'],
                    'core_promoter_id': core['ID'],
                    'tf_name': tfbm['tf_name'],
                    'tf_family': tfbm['tf_family'],
                    'motif_source': tfbm['motif_source'],
                    'copy_number': tfbm['copy_number'],
                    'helix_position': tfbm['helix_position'],
                    'core_promoter_group': core.get('Group', 'Unknown'),
                    'has_tata': core.get('Has_TATA', False),
                    'full_sequence': full_seq,
                    'total_length': len(full_seq),
                    'tfbm_length': tfbm['total_length'],
                    'core_length': core.get('Length', len(core['Sequence'])),
                    'expression_type': tfbm['expression_type'],
                    'expected_activity': tfbm['relative_activity'],
                })
        
        return pd.DataFrame(combined)


# ============================================================================
# Main Execution
# ============================================================================

def main():
    """メイン実行関数"""
    
    print("\n" + "="*80)
    print("🧬 TFBM Library Generator")
    print("   JASPAR & HT-SELEX Based Design System")
    print("="*80)
    print(f"\n📅 Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("\n【Design Principles】")
    print("  基準1: JASPAR (ユビキタス発現) + HT-SELEX (高発現予測)")
    print("  基準2: 4コピー / 6コピー")
    print("  基準3: 10 bp spacer (同一側面) / 5 bp spacer (反対側面)")
    print("="*80 + "\n")
    
    # ライブラリ生成
    generator = TFBMModuleGenerator()
    
    df = generator.generate_library(
        include_jaspar=True,
        include_htselex=True,
        htselex_min_enrichment=1.0
    )
    
    # サマリー表示
    summary = generator.get_summary(df)
    
    print("📊 Library Summary")
    print("-"*50)
    print(f"  Total modules: {summary['total_modules']}")
    print(f"  Unique TFs: {summary['unique_tfs']}")
    
    print(f"\n  By Source:")
    for source, count in summary['by_source'].items():
        print(f"    - {source}: {count}")
    
    print(f"\n  By Copy Number:")
    for copy, count in summary['by_copy_number'].items():
        print(f"    - {copy}x: {count}")
    
    print(f"\n  By Spacer Length (Helix Position):")
    for spacer, count in summary['by_spacer'].items():
        side = "same side" if spacer == 10 else "opposite side"
        print(f"    - {spacer} bp ({side}): {count}")
    
    print(f"\n  By Expression Type:")
    for exp, count in sorted(summary['by_expression_type'].items(), key=lambda x: -x[1]):
        print(f"    - {exp}: {count}")
    
    print(f"\n  By TF Family:")
    for family, count in sorted(summary['by_family'].items(), key=lambda x: -x[1])[:10]:
        print(f"    - {family}: {count}")
    
    print(f"\n  Length Statistics:")
    print(f"    - Min: {summary['length_stats']['min']} bp")
    print(f"    - Max: {summary['length_stats']['max']} bp")
    print(f"    - Mean: {summary['length_stats']['mean']} bp")
    
    print(f"\n  GC Content Statistics:")
    print(f"    - Min: {summary['gc_content_stats']['min']}%")
    print(f"    - Max: {summary['gc_content_stats']['max']}%")
    print(f"    - Mean: {summary['gc_content_stats']['mean']}%")
    
    # CSVに保存
    output_file = 'tfbm_library.csv'
    df.to_csv(output_file, index=False)
    print(f"\n📁 Output saved: {output_file}")
    
    # サンプル表示
    print("\n" + "="*80)
    print("📋 Sample Modules")
    print("="*80)
    
    # 各カテゴリからサンプル表示
    samples = [
        ('SP1', 4, 10),      # JASPAR, 4x, same side
        ('MYC', 6, 5),       # JASPAR, 6x, opposite side
        ('SRF', 4, 10),      # HT-SELEX, 4x, same side
        ('MAFB', 6, 5),      # HT-SELEX, 6x, opposite side
    ]
    
    for tf, copy, spacer in samples:
        sample = df[(df['tf_name'] == tf) & 
                    (df['copy_number'] == copy) & 
                    (df['spacer_length'] == spacer)]
        if not sample.empty:
            row = sample.iloc[0]
            side = "same side" if spacer == 10 else "opposite side"
            print(f"\n[{row['module_id']}]")
            print(f"  TF: {row['tf_name']} ({row['tf_family']})")
            print(f"  Source: {row['motif_source']}")
            print(f"  Config: {row['copy_number']}x copies, {row['spacer_length']} bp spacer ({side})")
            print(f"  Motif: {row['optimal_sequence']}")
            print(f"  Length: {row['total_length']} bp, GC: {row['gc_content']}%")
            seq = row['full_sequence']
            if len(seq) > 70:
                print(f"  Sequence: {seq[:70]}...")
            else:
                print(f"  Sequence: {seq}")
    
    # データベース情報
    print("\n" + "="*80)
    print("📚 Available Motifs in Databases")
    print("="*80)
    
    print("\n【JASPAR Motifs】")
    jaspar_db = JASPARMotifDatabase()
    for name in jaspar_db.list_available_motifs():
        motif = jaspar_db.get_motif(name)
        print(f"  {name}: {motif.optimal_sequence} ({motif.tf_family})")
    
    print("\n【HT-SELEX Motifs】")
    htselex_db = HTSELEXMotifDatabase()
    for name in htselex_db.list_available_motifs():
        motif = htselex_db.get_motif(name)
        enrich = f", enrich={motif.ht_selex_enrichment}" if motif.ht_selex_enrichment else ""
        print(f"  {name}: {motif.optimal_sequence} ({motif.tf_family}{enrich})")
    
    print("\n" + "="*80)
    print("✅ TFBM Library Generation Complete!")
    print("="*80)
    print("\n🔗 Integration with Core Promoter:")
    print("   >>> from tfbm_library_generator import IntegratedPromoterLibrary")
    print("   >>> lib = IntegratedPromoterLibrary()")
    print("   >>> lib.load_core_promoters('validated_core_promoters_26.csv')")
    print("   >>> tfbm = lib.generate_tfbm_library()")
    print("   >>> full = lib.combine_with_core_promoter(tfbm)")
    print("="*80 + "\n")
    
    return df


if __name__ == "__main__":
    df = main()
